In [ ]:
!pip3 install git+https://github.com/arogozhnikov/einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/arogozhnikov/einops to /tmp/pip-req-build-lyqht2l7
  Running command git clone -q https://github.com/arogozhnikov/einops /tmp/pip-req-build-lyqht2l7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


# Separable convolutions

![](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*o3mKhG3nHS-1dWa_plCeFw.png)

In [5]:
import torch
from torch import nn
from einops import rearrange


In [8]:
class SeparableDepthwise(nn.Module):
    def __init__(self, c, kernel_size):
        super(SeparableDepthwise, self).__init__()
    
        self.c = c
        self.kernel_size = kernel_size
        self.filters = nn.ModuleList(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=kernel_size)
            for _ in range(c)
        )
    
    def forward(self, x):
        channels = rearrange(x, 'b c h w -> c b 1 h w')
        new_channels = []
        for i in range(self.c):
            new_channels.append(self.filters[i](channels[i]))
        out = rearrange(new_channels, 'c b 1 h w -> b c h w')
        return out
      


class SeparableConv2d(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, bias=False):
      super(SeparableConv2d, self).__init__()
      self.depthwise = SeparableDepthwise(in_channels, kernel_size)
      self.pointwise = nn.Sequential(
         nn.Conv2d(kernel_size=1, in_channels=in_channels, out_channels=out_channels)
      )

  def forward(self, x):
      out = self.depthwise(x)
      out = self.pointwise(out)
      return out

In [11]:
features = torch.rand([1, 5, 25, 25])
print(features.size())
out = SeparableConv2d(5, 3, 3)
out(features).shape

torch.Size([1, 5, 25, 25])


torch.Size([1, 3, 23, 23])

# R(2+1) conv

https://www.tensorflow.org/tutorials/video/video_classification

https://paperswithcode.com/method/2-1-d-convolution

![alt text](https://drive.google.com/uc?export=view&id=1DDI_5xclb7wb1V2vtDzgoAKm2psjd1qb)

In [25]:
import torch
from torch import nn as nn
from einops.layers.torch import Rearrange

In [32]:
class R2_and1_conv(torch.nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size):
    super().__init__()
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size

    self.first_step = nn.Sequential(
      Rearrange('c t w h -> t c w h'),
      SeparableDepthwise(in_channels, kernel_size=(kernel_size[0], kernel_size[1])),
      Rearrange('t c w h -> c t w h')
    )
    
    self.second_step = nn.Sequential(
      # Это просто конв-слой размеров k*1*1
      nn.Conv3d(in_channels=in_channels, out_channels=out_channels, kernel_size=(kernel_size[2], 1, 1))
    )

    # first step: [c, t, h, w] -> [c, t, h1, w1]
    # second step: [c, t, h1, w1] -> [c, t2, h1, w1]

  def forward(self, x):
    out = self.first_step(x)
    out = self.second_step(out)
    return out

In [13]:
# [c, t, h, w]
x = torch.rand((1, 10, 5, 5))
conv3d = nn.Conv3d(in_channels=1, out_channels=1, kernel_size=3)

In [14]:
out_x = conv3d(x)
out_x.shape

torch.Size([1, 8, 3, 3])

In [33]:
out_x2 = R2_and1_conv(1, 1, [3, 3, 3])(x)
out_x2.shape

torch.Size([1, 8, 3, 3])

#Temporal attention

[GLTR](https://openaccess.thecvf.com/content_ICCV_2019/papers/Li_Global-Local_Temporal_Representations_for_Video_Person_Re-Identification_ICCV_2019_paper.pdf)

In [ ]:
T = 100 #10frames
d = 20 #inner dim size
input_features = torch.rand((d, T))

In [ ]:
class GLRT(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, f):
    pass

In [ ]:

GLRT()(input_features).shape

torch.Size([60, 1])